In [ ]:
# 思路：递归。结束条件分别是输入空则返回空，输入单元素列表则返回单节点，否则递归
# 先序序列的首元素是当前轮次的根节点，然后在中序序列中找到根节点的idx
# 划分先序序列与中序序列进行递归


def build_rec(preorder, inorder):
    if not preorder:
        return None
    elif len(preorder) == 1:
        return TreeNode(preorder[0])

    cur_root = TreeNode(preorder[0])    # 先序序列的首元素是根节点
    root_inorder_idx = inorder.index(cur_root.val)    # 找到根节点的中序索引

    # 左边是左子树，右边是右子树
    cur_root.left = build_rec(preorder[1:root_inorder_idx+1],
                              inorder[:root_inorder_idx])
    cur_root.right = build_rec(preorder[root_inorder_idx+1:],
                               inorder[root_inorder_idx+1:])
    return cur_root

给定一颗二叉树与一个节点，找出其中序序列的下一个节点，节点有三个指针，分别指向左子节点、右子节点和父节点。

In [2]:
class TreeNode:
    def __init__(self, x=None):
        self.val = x
        self.left = None
        self.right = None
        self.p = None

# 分情况讨论
# 有右子树
#     则找到右子树中最左的节点
# 无右子树
#     有父节点
#         该节点是父节点的左子树，则返回父节点
#         该节点是父节点的右子树，向上搜索找到最近的左子节点
#     无父节点
#         返回空
def next_node(node):
    if node is None:
        return None

    next_node = None  # 返回值置空

    if node.right is not None:  # 存在右子树
        node = node.right
        while node.left is not None:
            node = node.left
        next_node = node
    elif node.p is not None:  # 无右子树有父节点
        cur_node = node
        while cur_node.p is not None and cur_node == cur_node.p.right:  # 向上搜索最近的左子节点
            cur_node = cur_node.p
        next_node = cur_node.p

    return next_node

给定两棵树，判断后者是不是包含在前者中。空树不算子结构。

In [3]:
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None


# 判断以root2为根节点的树是否包含在以root1为根节点的树中
def has_substruct(root1, root2):
    if root1 is None or root2 is None:  # 左右任一根节点空则失匹，因为空树不算子结构
        return False
    if root1.val != root2.val:
        return False
    else:
        # 首先从根节点判断是否左边包含右边
        return (contain_rec(root1, root2) or
                has_substruct(root1.left, root2) or
                has_substruct(root1.right, root2))


# 递归判断左节点是否包含右节点，并且空树的处理由has_substruct()完成
# 所以这里当右节点为空时应该返回True
def contain_rec(node1, node2):
    if node2 is None:
        return True
    if node1 is None or node1.val != node2.val:
        return False
    return contain_rec(node1.left, node2.left) and contain_rec(node1.right, node2.right)

给定一颗二叉树，输出其按照根节点左右翻转的镜像。

In [4]:
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None


class Stack:
    def __init__(self):
        self.items = []
        self.size = 0

    def push(self, x):
        self.items.insert(0, x)
        self.size += 1

    def pop(self):
        self.size -= 1
        return self.items.pop()


# 使用循环与栈来模拟递归
# 先将根节点入栈，然后类似层次遍历，左右子节点入栈
# 每弹出一个节点，交换其左右子节点
def mirror_tree(root_node):
    if root_node is None:  # 空树判断
        return root_node

    stack = Stack()
    stack.push(root_node)  # 根节点入栈

    while stack.size > 0:
        node = stack.pop()
        node.left, node.right = node.right, node.left  # 镜像翻转子节点
        # 再把子节点入栈
        if node.left:
            stack.push(node.left)
        if node.right:
            stack.push(node.right)

二叉树的层次遍历。

In [6]:
# 需要队列来实现，每访问一个节点就将其子节点加入队列

def LevelTrav(root_node):
    if not root_node:
        return []

    queue = [root_node]
    res = []

    while queue:
        cur_node = queue.pop(0)
        res.append(cur_node.val)
        if cur_node.left:
            queue.append(cur_node.left)
        if cur_node.right:
            queue.append(cur_node.right)

    return res

二叉搜索树的后序遍历合法性判断。给定一个无重复的序列，判断是否是一颗二叉搜索树的后序遍历序列。假设空序列为False。

In [9]:
# 如果某序列是二叉搜索树的后序遍历，那么根节点之前的序列可分为两部分，左子树(小于根节点)和右子树(大于根节点)
def VerifySquenceOfBST(s):
    if not s:
        return True

    root_val = s[-1]

    # 找到左子树的边界位置
    for i in range(len(s)):
        if s[i] > root_val:
            break

    # 判断右子树是否都大于根节点
    for j in range(i, len(s)):
        if s[j] < root_val:
            return False

    # 把左右标记默认设为True，对应空子树的情况，当左右子树的元素数量>0时才做递归判断
    left_flag = True
    right_flag = True
    
    if i > 0:
        left_flag = VerifySquenceOfBST(s[:i])
    if i < len(s)-1:
        right_flag = VerifySquenceOfBST(s[i:-1])

    return left_flag and right_flag

二叉树中的求和路径。给定一个数字，求出二叉树中满足和等于该值的路径。

In [10]:
# 思路，递归，逆推法。
# 使用先序遍历访问所有可能的节点，同时更新差值
# 当叶子节点值等于剩余差值时，记录该叶节点，并向上回溯


def SumPath(root_node, num):
    if not root_node:
        return []

    # 单节点，并等于差值时
    if root_node and not root_node.left and not root_node.right and root_node.val == num:
        return [[root_node.val]]    # 返回一个单节点路径

    # 向左探索，并更新差值，直到叶子节点
    left_path = SumPath(root_node.left, num -
                        root_node.val) if root_node.left else []
    # 向右探索，更新差值，直到叶子节点
    right_path = SumPath(root_node.right, num -
                         root_node.val) if root_node.right else []

    res = []
    for path in left_path+right_path:
        res.append([root_node.val]+path)    # 把探索到的合法路径拼到当前节点后面，再加入返回列表
    return res

二叉搜索树转有序双向链表。

In [11]:
# 思路：二叉搜索树可递归地分为两部分，左分支，右分支；转换后的链表也可递归地分为两部分
# 使用中序遍历来保证有序


def ST2DL(tree_root):
    # 空节点判断
    if not tree_root:
        return None
    # 叶节点判断
    if not tree_root.left and not tree_root.right:
        return tree_root

    left_root = ST2DL(tree_root.left)   # 往左递归直到遇到左空或左叶
    if left_root:    # 存在左叶
        # 找到左边部分的最大值
        cur_max = left_root
        while cur_max and cur_max.right:
            cur_max = cur_max.right

        # 把root与左最大值转成双链表
        cur_max.right = tree_root
        tree_root.left = cur_max

    # 右分支
    right_root = ST2DL(tree_root.right)
    if right_root:
        cur_min = right_root
        while cur_min and cur_min.left:
            cur_min = cur_min.left

        cur_min.left = tree_root
        tree_root.right = cur_min

    return left_root if left_root else tree_root    # 返回链表头结点

二叉搜索树中第k大的节点。

In [13]:
# 中序遍历，维护一个充当计数器的全局变量即可
cnt = 0


def KthNodeInST(root_node, k):
    if root_node:    # 节点存在
        res = KthNodeInST(root_node.left, k)    # 往左走
        if res:    # 在左分支得到了答案则直接返回
            return res

        # 判断当前节点是否是需要搜索的点
        cnt += 1
        if cnt == k:    # 找到了直接返回
            return root_node

        res = KthNodeInST(root_node.right, k)    # 往右走
        if res:
            return res
    else:
        return None

Balanced Binary Tree. 判别平衡二叉树。

In [15]:
# 思路：后序遍历
# 每个递归函数返回以输入节点为根的子树深度，当子树不是平衡树时返回特殊值-1

def isBalanced(root):
    return self.isBalanced_rec(root)!=-1

def isBalanced_rec(root):
    if not root:    # 叶子节点返回0
        return 0

    # 左子树不平衡返回-1
    if isBalanced_rec(root.left)==-1:
        return -1
    # 右子树不平衡返回-1
    if isBalanced_rec(root.right)==-1:
        return -1

    # 如果左右子树高度差小于2，则返回最大深度；否则返回-1
    return max(left_dep,right_dep)+1 if abs(left_dep-right_dep)<2 else -1

Find Mode in Binary Search Tree. 求二叉搜索树中的众数。

In [16]:
# 思路：二叉搜索树的中序遍历为排序数组，该题可转化成在排序数组中查找众数


def findMode(root: TreeNode) -> List[int]:
    if not root:
        return []

    # 全局状态变量
    max_cnt = 0
    cur_cnt = 0
    res = list()
    pre = None

    MidTrav(root)

    return res


def MidTrav(root):
    if not root:    # 空判定，防止传入空值
        return None

    nonlocal max_cnt
    nonlocal cur_cnt
    nonlocal res
    nonlocal pre

    # 往左走
    MidTrav(root.left)

    if root.val == pre:
        cur_cnt += 1
    else:
        cur_cnt = 1
        pre = root.val

    if cur_cnt > max_cnt:
        res = [root.val]    # 重新赋值
        max_cnt = cur_cnt
    elif cur_cnt == max_cnt:
        res.append(root.val)

SyntaxError: no binding for nonlocal 'max_cnt' found (cell_name, line 26)

Binary Tree Level Order Traversal. 二叉树的层次遍历，按层级输出。

In [ ]:
# 思路：正常的层次遍历，使用队列实现
# 要实现按层级输出，需要在内循环内在设置一个循环对每层做单独处理


def levelOrder(root):
    if not root:
        return []

    q = list()
    q.append(root)
    res=[]

    while q:
        level_size = len(q)    # 当前层的节点数
        cur_level = [None]*level_size    # 设置临时数组以整体形式保存当前层的节点

        # 同一层级的遍历
        for idx in range(level_size):
            cur_node = q.pop(0)
            cur_level[idx]=cur_node.val
            
            # 子节点入队
            if cur_node.left:
                q.append(cur_node.left)
            if cur_node.right:
                q.append(cur_node.right)
        
        res.append(cur_level)
        
    return res

Binary Tree Level Order Traversal II. 二叉树的逆层级遍历，按层级输出。

In [ ]:
# 思路：正常的层序遍历，不过把结果压入栈，然后再反转
# 一个队列实现正常的层次遍历，栈实现结果的反转


def levelOrderBottom(self, root):
    if not root:
        return []

    q = list()
    s = list()
    q.append(root)

    while q:
        # 对当前层节点计数
        level_size = len(q)
        cur_level = [None]*level_size

        # 同一层次内的操作
        for idx in range(level_size):
            cur_node = q.pop(0)
            cur_level[idx] = cur_node.val

            if cur_node.left:
                q.append(cur_node.left)
            if cur_node.right:
                q.append(cur_node.right)

        s.append(cur_level)    # 该层所有节点入栈

    res = []
    while s:    # 弹栈
        res.append(s.pop())

    return res

[Binary Tree Zigzag Level Order Traversal](https://leetcode.com/problems/binary-tree-zigzag-level-order-traversal/)。二叉树的层级遍历，按之字形输出。

In [ ]:
# 思路：记录层数，按层数选择输出方式


def zigzagLevelOrder(root):
    if not root:
        return list()

    res = list()
    q = [root]
    even_level = True    # 偶数层标志

    while q:
        level_size = len(q)
        cur_level = [None]*level_size
        
        for idx in range(level_size):
            cur_node = q.pop(0)
            cur_level[idx] = cur_node.val

            if cur_node.left:
                q.append(cur_node.left)
            if cur_node.right:
                q.append(cur_node.right)

        if not even_level:
            res.append(list(reversed(cur_level)))
        else:
            res.append(cur_level)
        even_level = not even_level

    return res

Minimum Depth of Binary Tree. 二叉树的最小深度，令根节点深度为1。

In [ ]:
# 思路：层次遍历，一旦遇到叶节点立马返回深度


def minDepth(root):
    if not root:
        return 0

    q = list()
    q.append(root)
    dep = 0

    while q:
        dep += 1
        level_size = len(q)

        for _ in range(level_size):    # 单层遍历
            cur_node = q.pop(0)

            # 碰到叶节点立即返回
            if not cur_node.left and not cur_node.right:
                return dep

            if cur_node.left:
                q.append(cur_node.left)
            if cur_node.right:
                q.append(cur_node.right)

Maximum Depth of Binary Tree. 二叉树的最大深度，令根节点深度为1.

In [ ]:
# 思路，进行一次完整的层次扫描


def maxDepth(root):
    if not root:
        return 0

    q = list()
    q.append(root)
    dep = 0

    while q:
        dep += 1
        level_size = len(q)

        for _ in range(level_size):
            cur_node = q.pop(0)

            if cur_node.left:
                q.append(cur_node.left)
            if cur_node.right:
                q.append(cur_node.right)

    return dep

[Merge Two Binary Trees](https://leetcode.com/problems/merge-two-binary-trees/). 合并两二叉树，同样位置的节点用和来覆盖。

In [ ]:
# 思路：本质上还是树的遍历，不过遍历时要将两棵树对应节点作为一个节点组来看待
# 以下代码将右树合并到左树中


def mergeTrees(t1, t2):
    if not t1:
        return t2

    s = []
    s.append((t1, t2))    # 压根节点组

    while s:
        cur_node = s.pop()    # 访问节点组
        if not cur_node[0] or not cur_node[1]:
            continue

        cur_node[0].val += cur_node[1].val    # 当前节点的合并

        if not cur_node[0].left:    # 如果左树的左子节点为空，则用右树的节点覆盖
            cur_node[0].left = cur_node[1].left
        else:
            s.append((cur_node[0].left, cur_node[1].left))

        if not cur_node[0].right:
            cur_node[0].right = cur_node[1].right
        else:
            s.append((cur_node[0].right, cur_node[1].right))

    return t1

[Univalued Binary Tree](https://leetcode.com/problems/univalued-binary-tree/). 判断一颗二叉树中所有节点值是否相同。

In [ ]:
# 思路：记录根节点的值，然后遍历，下述代码使用先序遍历
# 先序遍历：根左右，先访问根节点，然后右节点入栈，再左节点入栈


def isUnivalTree(root):
    if not root:
        return True

    val = root.val
    s = list()
    s.append(root)

    while s:
        cur_node = s.pop()

        if cur_node.val != val:
            return False

        if cur_node.right:
            s.append(cur_node.right)
        if cur_node.left:
            s.append(cur_node.left)

    return True

[N-ary Tree Postorder Traversal](https://leetcode.com/problems/n-ary-tree-postorder-traversal/). 多叉树的后序遍历。

In [ ]:
# 思路：非递归实现后序遍历是比较难的，想办法先得到一个逆序的后序遍历
# 后序遍历顺序为左右根，使用根右左的遍历顺序得到一个遍历序列，再反转即可

def postorder(root: 'Node') -> List[int]:
    if not root:
        return []
    
    s=list()
    s.append(root)
    res=list()
    
    while s:
        cur_node=s.pop()
        res.append(cur_node.val)
        
        # 子节点以列表存在，从左往右依次入栈，可用语法糖
        s.extend(cur_node.children)
                
    # 将遍历序列反转
    return res[::-1]

[N-ary Tree Preorder Traversal](https://leetcode.com/problems/n-ary-tree-preorder-traversal/). 多叉树的先序遍历。

In [ ]:
# 思路：先序遍历的顺序为根左右，那么入栈顺序就为右左

def preorder(root: 'Node') -> List[int]:
    if not root:
        return []
    
    s=list()
    s.append(root)
    res=[]
    
    while s:
        cur_node=s.pop()
        res.append(cur_node.val)
        
        # 从右往左入栈，可用语法糖
        s.extend(reversed(cur_node.children))
            
    return res

[Maximum Depth of N-ary Tree](https://leetcode.com/problems/maximum-depth-of-n-ary-tree/). 求多叉树的深度。

In [ ]:
# 思路，层次遍历


def maxDepth(root) -> int:
    if not root:
        return 0

    q = list()
    q.append(root)
    dep = 0

    while q:
        level_size = len(q)
        dep += 1

        for _ in range(level_size):
            cur_node = q.pop(0)

            if cur_node.children:
                q.extend(cur_node.children)

    return dep

[Kth Smallest Element in a BST](https://leetcode.com/problems/kth-smallest-element-in-a-bst/)。

In [ ]:
# 思路，中序遍历的同时计数，计数值到达一定时返回


def kthSmallest(root, k: int) -> int:
    s = []
    while root or s:
        while root:
            s.append(root)
            root = root.left

        root = s.pop()
        k -= 1

        if k == 0:
            return root.val
        root = root.right

[Populating Next Right Pointers in Each Node](https://leetcode.com/problems/populating-next-right-pointers-in-each-node/)。一颗满二叉树，将同层次的节点使用next指针连接起来，最右边的节点置空。

In [ ]:
class Node:
    def __init__(self, val, left, right, next):
        self.val = val
        self.left = left
        self.right = right
        self.next = None

# 思路：层次遍历，因为是满二叉树，所以左子节点必定存在next节点
# 而右子节点是否存在next节点需要判断父节点是否存在next节点


def connect(root: 'Node') -> 'Node':
    if not root:
        return root

    q = [root]

    while q:
        level_size = len(q)

        for idx in range(level_size):    # 同层遍历
            cur_node = q.pop(0)

            if cur_node.left:    # 存在左子节点必定存在右子节点
                cur_node.left.next = cur_node.right    # 左子节点的next为右子节点
                if cur_node.next:
                    cur_node.right.next = cur_node.next.left    # 右子节点的next为父节点next的左子节点

                q.append(cur_node.left)
                q.append(cur_node.right)

    return root